In [1]:
%pip install pdfplumber
%pip install --upgrade openai --quiet
%pip install langchain_nvidia_ai_endpoints
%pip install -U langchain-nvidia-ai-endpoints
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

import requests
import pdfplumber
import re
from openai import OpenAI
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
import faiss
from ipywidgets import interact_manual, Text, Output
import IPython.display as display

In [3]:
# Importing the NVIDIA API KEY from Kaggle 

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("Nvapi_Key")


# Passing to the Client for Embedding Purpose

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=secret_value_1
)

# Chunking We using the Jina AI

secret_value_0 = user_secrets.get_secret("Jina_key")


# Importing The PDF

In [4]:
# Importing PDF Using pdfplumber

with pdfplumber.open('/kaggle/input/insurance-final/ICICI_Insurance.pdf') as pdf:
    all_text=''
    for page in pdf.pages:
        all_text += page.extract_text()

print(all_text[:100])
    

A Plan to Fit Your Needs
You constantly strive to provide your loved ones with a future
full of happ


In [5]:
# 1. Cleaning of The PDF

def clean_pdf(text):

    text = re.sub(r'Page \d+\n', '', text)  # Remove page numbers
    text = re.sub(r'ICICI .*?\n', '', text) # Remove ICICI header/footer
    text = re.sub(r'IRDAI Regn.*?\n', '', text) # Remove regulator lines
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text) # Fix hyphenated words
    text = re.sub(r'[\u2022•]', '-', text) # Replace bullets with dash
    text = re.sub(r'(?<![.!?])\n', ' ', text) # Join lines that don't end with punctuation
    text = re.sub(r'\n+', '\n', text) # Remove extra newlines
    text = re.sub(r' {2,}', ' ', text) # Remove extra spaces
    return(text)

# Implementing the Function on the All_text

pdf_cleaned = clean_pdf(all_text)
print(pdf_cleaned[:1000])

A Plan to Fit Your Needs You constantly strive to provide your loved ones with a future full of happiness and comfort. However, life isn’t always predictable. Sometimes, unexpected events can stand in the way of your dreams.
Presenting exclusively for you to help provide financial security for your family, no matter what. It safeguards your family’s future against life's uncertainties and also gives you two plan options as per your need.A Plan to Fit Your Needs A Plan to Fit Your Needs Life Insurance Cover Choice of two plan options Flexible policy term for financial security - With Return of Premium and premium of your family - Without Return of Premium payment options Life Stage Protection feature to enhance your Sum Assured 15% lower premiums Special discounts on key milestones for women customers for salaried customers in the Life Assured’s life like marriage, childbirth or disbursement of Home loan Tax benefits may be applicable on premiums paid and benefits received as per the pr

# Importing The ICICI WebSite

In [6]:
# Importing the ICICI Website

def read_website(url, secret_value_0):
    # Jina Reader expects the format: https://r.jina.ai/<full-url>
    api_url = f"https://r.jina.ai/{url}"
    headers = {"Authorization": f"Bearer {secret_value_0}"}
    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Failed to read website: {response.status_code}, {response.text}")

url = "https://www.icicibank.com/personal-banking/insurance?ITM=nli_home_na_megamenuItem_1CTA_CMS_insurance_viewAllInsurance_NLI"
website_text = read_website(url, secret_value_0)

print(website_text[:500])  # Just show first 1000 chars

Title: ICICI Bank Insurance Plans - Secure Your Future Today

URL Source: https://www.icicibank.com/personal-banking/insurance?ITM=nli_home_na_megamenuItem_1CTA_CMS_insurance_viewAllInsurance_NLI

Markdown Content:
ICICI Bank Insurance Plans - Secure Your Future Today


[![Image 1: ICICI Bank](https://www.icicibank.com/content/dam/icicibank-revamp/images/icici-logo/icici-header-logo.png)ICICI Bank](https://www.icicibank.com/home?ITM=nli_insurance_insurance_headerComponent_1CTA_CM


# Combining the PDF and WebSite (WebSite & PDF)

In [7]:
combined_text = (
    "STATIC PDF DATA:" +"\n" +pdf_cleaned +
    "\n\nWEBSITE DATA \n" + website_text)
combined_text[:500]

"STATIC PDF DATA:\nA Plan to Fit Your Needs You constantly strive to provide your loved ones with a future full of happiness and comfort. However, life isn’t always predictable. Sometimes, unexpected events can stand in the way of your dreams.\nPresenting exclusively for you to help provide financial security for your family, no matter what. It safeguards your family’s future against life's uncertainties and also gives you two plan options as per your need.A Plan to Fit Your Needs A Plan to Fit You"

In [8]:
# 2. Chunking the text Using the Jina AI

JINA_SEGMENT_URL = "https://api.jina.ai/v1/segment"

def chunk_with_jina(text: str, max_len: int = 8000):
    """Split text into <=8k chunks and send each to Jina Segmenter API with fallback."""
    headers = {"Authorization": f"Bearer {secret_value_0}"}
    chunks = []

    for i in range(0, len(text), max_len):
        part = text[i:i+max_len]

        payload = {
            "content": part,
            "config": {"split_length": 500}  # ~500 char chunks
        }

        resp = requests.post(JINA_SEGMENT_URL, headers=headers, json=payload)

        if resp.status_code == 200:
            data = resp.json()
            segs = [seg["text"] for seg in data.get("segments", [])]

            if segs:
                chunks.extend(segs)
            else:
                # fallback: just append raw piece if Jina failed
                chunks.append(part)

        else:
            print("Jina Segmenter failed:", resp.status_code, resp.text)
            chunks.append(part)  # fallback

    return chunks


chunk = chunk_with_jina(combined_text)
print(len(chunk))

106


In [9]:
# 3. Embedding the Chunk Using the Jina AI Embedder


JINA_EMBED_URL = "https://api.jina.ai/v1/embeddings"

import numpy as np
import requests

JINA_EMBED_URL = "https://api.jina.ai/v1/embeddings"

def embed_with_jina(texts, api_key, batch_size=16):
    """
    Convert a list of texts into embeddings using Jina Embeddings API.
    Splits into batches to avoid timeout / payload errors.
    """
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    all_vectors = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = [{"text": t} for t in batch]

        payload = {
            "model": "jina-embeddings-v4",
            "task": "text-matching",   # good for chatbot/retrieval
            "input": inputs
        }

        resp = requests.post(JINA_EMBED_URL, headers=headers, json=payload)

        if resp.status_code == 200:
            data = resp.json()
            vectors = [item["embedding"] for item in data["data"]]
            all_vectors.extend(vectors)
        else:
            raise Exception(f"Embedding failed: {resp.status_code}, {resp.text}")

    return np.array(all_vectors, dtype="float32")


 


embeds = embed_with_jina(chunk, secret_value_0)
print(embeds)

[[-0.01428223 -0.04199219  0.05932617 ... -0.00772095  0.01098633
   0.02075195]
 [-0.01574707 -0.02001953  0.04370117 ... -0.00125885  0.01501465
   0.0213623 ]
 [-0.00805664 -0.02563477  0.02783203 ... -0.01434326  0.01074219
   0.01831055]
 ...
 [-0.01855469 -0.03027344 -0.00585938 ...  0.0062561   0.02246094
   0.00341797]
 [-0.01428223 -0.03222656  0.00549316 ... -0.00224304  0.01483154
   0.00753784]
 [-0.02661133 -0.01843262 -0.01092529 ...  0.00817871  0.02307129
   0.00241089]]


In [10]:
# 4. Indexing By Using FAISS(Facebook AI Similarity Search)

def build_index(vectors):
    faiss.normalize_L2(vectors)
    index = faiss.IndexFlatIP(vectors.shape[1])
    index.add(vectors)
    return index

index = build_index(embeds)
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7fd48f933960> >

In [11]:
# 5. Searching fuction Using Jina AI (User Entered Query into Vectors)

def embed_query_jina(query, api_key):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "jina-embeddings-v4",
        "task": "text-matching",
        "input": [{"text": query}]
    }
    resp = requests.post(JINA_EMBED_URL, headers=headers, json=payload)

    if resp.status_code == 200:
        data = resp.json()
        vector = data["data"][0]["embedding"]
        return np.array([vector], dtype="float32")   # shape (1, dim)
    else:
        raise Exception(f"Query embedding failed: {resp.status_code}, {resp.text}")


In [12]:
# 6. Searching the Query by User

def search(query, index, chunk, top_k=3):
    q_vec = embed_query_jina(query, secret_value_0)  # Use correct function
    faiss.normalize_L2(q_vec)
    D, I = index.search(q_vec, top_k)
    return [chunk[i] for i in I[0]]

In [13]:
# 7. Reranking the Document for Better and Accurate Results

def rerank_with_jina(query, docs, api_key, top_n=3):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    formatted_docs = [{"text": d} for d in docs]

    data = {
        "model": "jina-reranker-v2-base-multilingual",
        "query": query,
        "top_n": top_n,
        "documents": formatted_docs,
        "return_documents": True
    }

    resp = requests.post("https://api.jina.ai/v1/rerank", headers=headers, json=data)
    out = resp.json()

    if resp.status_code == 200 and "results" in out:
        return [r["document"]["text"] for r in out["results"]]
    else:
        raise Exception(f"Reranking failed: {resp.status_code}, {out}")


In [19]:
# 8. Now, Answering the Question

def answer_question(query, index, chunk, history, api_key, top_k=5, rerank_n=3):
    # Step 1: Retrieve candidates
    retrieved = search(query, index, chunk, top_k=top_k)

    # Step 2:  To do with Reranker
    reranked = rerank_with_jina(query, retrieved, api_key, top_n=rerank_n)

    # Step 3: Build context
    context = "\n".join(reranked)

    # Step 4: Use LLM to answer
    resp = client.chat.completions.create(
        model="nvidia/llama-3.1-nemotron-70b-instruct",
        messages=[
            {"role": "system", "content": "You are an insurance assistant."},
            *history,
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
        ]
    )

    answer = resp.choices[0].message.content

    # Update history
    history.append({"role": "user", "content": query})
    history.append({"role": "assistant", "content": answer})

    return answer

In [20]:
# 9. Defining the Intent of the User

def classify_intent(query):
    q = query.lower().strip()

    
    if q in ["hi", "hello", "hey", "good morning", "good evening"]:
        return "greeting"

    elif any(word in q for word in ["coverage", "benefits", "exclusions", "claim", "policy", "premium", "health insurance"]):
        return "policy_info"

    elif any(word in q for word in ["company", "about", "vision", "history", "branches", "icici"]):
        return "company_info"

    elif any(word in q for word in ["contact", "support", "customer care", "help", "payment"]):
        return "general_support"

    else:
        return "policy_info"




In [21]:
# 10. Remembering the Past Memory (Old Conversation)

def insurance_chatbot(query, chunk, index, company_kb, faq_kb, history):
    intent = classify_intent(query)

    if intent == "greeting":
        return "Hello 👋! I'm your insurance assistant. How can I help you today?"

    elif intent == "policy_info":
        return answer_question(query, index, chunk, history,secret_value_0)

    elif intent == "company_info":
        return company_kb.get("about_company", "Company information not available.")

    elif intent == "general_support":
        return faq_kb.get(query.lower(), "Please contact customer support at 1800-209-9777.")

    else:
        return "I'm here to help with insurance-related queries. Could you rephrase?"




In [22]:
# Converting into Real-time ChatBot

company_kb = {
    "about_company": "ICICI Prudential Life Insurance is one of the leading insurance providers in India, "
                     "offering term plans, health insurance, and savings plans."
}
faq_kb = {
    "how can i contact customer support?": "You can call our toll-free number 1800-209-9777 or email support@iciciprulife.com",
    "how do i pay premium?": "Premiums can be paid online via netbanking, debit/credit card, or through ICICI branches."
}

# Start conversation
history = []


In [23]:
# Start chatbot conversation loop
history = []

print("Insurance Chatbot is ready! (type 'exit' to stop)\n")

while True:
    query = input("You: ")  # type question
    if query.lower() in ["exit", "quit", "bye"]:
        print("Bot: Thank you for chatting. Have a great day! ")
        break

    response = insurance_chatbot(query, chunk, index, company_kb, faq_kb, history)
    print("Bot:", response, "\n")

Insurance Chatbot is ready! (type 'exit' to stop)



You:  what are the insuarnce policies for students


Bot: Based on the provided context, which appears to be an Accidental Death and Disability Rider policy, I'll extract and interpret the information to address your question about insurance policies for students. Please note that the original context doesn't explicitly mention "students," so I'll provide a tailored response by making reasonable assumptions.

**Assumptions:**

1. **Age Range**: Students are typically between 16 and 25 years old (high school to undergraduate studies).
2. **Relevance**: Policies that could be relevant or beneficial to students, considering their lifestyle and common risks.
3. **Exclusions**: Certain exclusions might not directly apply to students if they don't engage in the specified activities (e.g., military operations, professional sports).

**Insurance Policy Provisions Relevant to Students:**

Based on the provided Accidental Death and Disability Rider, here are some provisions that could be relevant or beneficial for students:

1. **Accidental Death 

You:  i am patient at a hosiptal what are benfits i recevice


Bot: Based on the provided context, which appears to be an insurance policy document outlining various medical conditions and their corresponding benefits, I'll extract the benefits that a patient (likely a female, given the context) might receive, assuming they are hospitalized and meet the specified conditions. Please note that:

1. **Eligibility**: Benefits are subject to the patient's policy terms, conditions, and the date of commencement of risk or reinstatement (if any).
2. **Condition-Specific Benefits**: Only benefits directly related to the conditions mentioned in the context are listed below. Other hospitalization benefits (e.g., room rent, doctor fees, etc.) might be part of the overall policy but are not explicitly mentioned here.
3. **Assumptions**:
	* The patient is a policyholder or an insured under the policy.
	* The conditions are diagnosed and treated by a specialist medical practitioner.
	* All specified conditions for each benefit are met.

**Potential Benefits for 

...

Bot: Here is a concise summary:

**Summary in Few Words**

* **Policy Exclusions (12 points)**: Suicide, Congenital Issues, War, Crimes, Hazardous Activities, etc.
* **Accidental Disability Benefit Exclusions (10 points)**: Similar to above, with additional exclusions for Pre-existing Diseases and Nuclear/Biological Attacks.
* **Accidental Death Cover Exclusions (4 points)**: Suicide, Congenital Issues, War, and Crimes.
* **Benefit Conditions for Various Health Issues (e.g., Cancer, Stroke, Eclampsia, Thyroid Storm)**: Specific diagnostic and treatment requirements for claim eligibility.
* **Activities of Daily Living (ADL) for Disability Assessment (6 points)**: Washing, Dressing, Transferring, etc. 



You:  is there any which support the suicide 


Bot: Please contact customer support at 1800-209-9777. 



You:  bye


Bot: Thank you for chatting. Have a great day! 
